## Azure OpenAI GPT4-o Target Demo
This notebook demonstrates how to use the Azure OpenAI GPT4-o target to accept multimodal input (text+image) and generate text output.

In [1]:
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT license.


from pyrit.models import PromptRequestPiece, PromptRequestResponse
from pyrit.prompt_target import AzureOpenAIGPT4OChatTarget
from pyrit.common import default_values
import pathlib
from pyrit.common.path import HOME_PATH
import uuid

default_values.load_default_env()
test_conversation_id = str(uuid.uuid4())

# use the image from our docs
image_path = pathlib.Path(HOME_PATH) / "assets" / "pyrit_architecture.png"

request_pieces = [
    PromptRequestPiece(
        role="user",
        conversation_id=test_conversation_id,
        original_value="Describe this picture:",
        original_value_data_type="text",
        converted_value_data_type="text",
    ),
    PromptRequestPiece(
        role="user",
        conversation_id=test_conversation_id,
        original_value=str(image_path),
        original_value_data_type="image_path",
        converted_value_data_type="image_path",
    ),
]

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
prompt_request_response = PromptRequestResponse(request_pieces=request_pieces)

In [3]:
with AzureOpenAIGPT4OChatTarget() as azure_openai_chat_target:
    resp = await azure_openai_chat_target.send_prompt_async(prompt_request=prompt_request_response)  # type: ignore
    print(resp)

None: assistant: The image is a structured chart detailing the components of PyRIT with two primary columns: "Interface" and "Implementation."

- **Target**:
  - Local: local model (e.g., ONNX)
  - Remote: API or web app

- **Datasets**:
  - Static: Prompts
  - Dynamic: Prompt templates

- **Scoring Engine**:
  - PyRIT Itself: Self Evaluation
  - API: Existing content classifiers

- **Attack Strategy**:
  - Single Turn: Using static prompts
  - Multi Turn: Multiple conversations using prompt templates

- **Memory**:
  - Storage: JSON, Database
  - Utils: Conversation, retrieval and storage, memory sharing, data analysis


## Multimodal Demo using AzureOpenAIGPT4OChatTarget and PromptSendingOrchestrator
This demo showcases the capabilities of AzureOpenAIGPT4OChatTarget for generating text based on multimodal inputs, including both text and images using PromptSendingOrchestrator.

In [4]:
from pyrit.common import default_values
import pathlib
from pyrit.common.path import HOME_PATH

from pyrit.prompt_target import AzureOpenAIGPT4OChatTarget
from pyrit.prompt_normalizer.normalizer_request import NormalizerRequestPiece
from pyrit.prompt_normalizer.normalizer_request import NormalizerRequest
from pyrit.orchestrator import PromptSendingOrchestrator

default_values.load_default_env()

azure_openai_gpt4o_chat_target = AzureOpenAIGPT4OChatTarget()

image_path = pathlib.Path(HOME_PATH) / "assets" / "pyrit_architecture.png"
data = [
    [
        {"prompt_text": "Describe this picture:", "prompt_data_type": "text"},
        {"prompt_text": str(image_path), "prompt_data_type": "image_path"},
    ],
    [{"prompt_text": "Tell me about something?", "prompt_data_type": "text"}],
    [{"prompt_text": str(image_path), "prompt_data_type": "image_path"}],
]

Construct list of NormalizerRequest objects

In [5]:

normalizer_requests = []

for piece_data in data:
    request_pieces = []

    for item in piece_data:
        prompt_text = item.get("prompt_text", "")  # type: ignore
        prompt_data_type = item.get("prompt_data_type", "")
        converters = []  # type: ignore
        request_piece = NormalizerRequestPiece(
            prompt_value=prompt_text, prompt_data_type=prompt_data_type, request_converters=converters  # type: ignore
        )
        request_pieces.append(request_piece)

    normalizer_request = NormalizerRequest(request_pieces)
    normalizer_requests.append(normalizer_request)

In [6]:
len(normalizer_requests)

3

In [7]:

with PromptSendingOrchestrator(prompt_target=azure_openai_gpt4o_chat_target) as orchestrator:

    await orchestrator.send_normalizer_requests_async(prompt_request_list=normalizer_requests)  # type: ignore

    memory = orchestrator.get_memory()

    for entry in memory:
        print(entry)

AzureOpenAIGPT4OChatTarget: user: E:\OSS_Tools\PyRIT-internal\PyRIT\assets\pyrit_architecture.png
AzureOpenAIGPT4OChatTarget: assistant: The image provides a detailed breakdown of the components of PyRIT, an AI-based system. Here is a summary of each component and its implementation:

### Interface: 
#### Target
- **Local:** local model (e.g., ONNX)
- **Remote:** API or web app

### Datasets
- **Static:** prompts
- **Dynamic:** Prompt templates

### Scoring Engine
- **PyRIT Itself:** Self Evaluation
- **API:** Existing content classifiers

### Attack Strategy
- **Single Turn:** Using static prompts
- **Multi Turn:** Multiple conversations using prompt templates

### Memory
- **Storage:** JSON, Database
- **Utils:** Conversation, retrieval and storage, memory sharing, data analysis

Each section defines key functionalities and methods that PyRIT leverages for processing and interaction within its framework.
AzureOpenAIGPT4OChatTarget: user: Tell me about something?
AzureOpenAIGPT4OChatT